In [4]:
from qiskit import IBMQ, QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit.visualization import plot_state_city, plot_histogram
from qiskit.providers.aer import noise
%matplotlib inline
import copy
import math
import numpy as np
# import qiskit
from plotTools import plot
import functools
import itertools as it


import json, glob

In [39]:
provider = IBMQ.load_account()
provider.backends()
device = provider.get_backend('ibmq_16_melbourne')
properties = device.properties()
coupling_map = device.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(properties)
basis_gates = noise_model.basis_gates
    

In [148]:
def qft(circuit):
    """
    Input: QuantumCircuit
    Returns: QFT circuit to be performed on the qregister
    """
    from qiskit.aqua.components.qfts.standard import Standard as StandardQFT

    QFT = StandardQFT(circuit.n_qubits)
    qft = QFT.construct_circuit(mode="circuit", qubits=circuit.qubits, circuit=circuit, do_swaps=False)

    return qft

def iqft(circuit):
    """
    Input: QuantumCircuit
    Returns: IQFT circuit to be performed on the qregister
    """
    from qiskit.aqua.components.iqfts.standard import Standard as StandardIQFT

    IQFT = StandardIQFT(circuit.n_qubits)
    iqft = IQFT.construct_circuit(mode="circuit", qubits=circuit.qubits, circuit=circuit, do_swaps=False)

    return iqft

In [238]:
def qubits_necessary(N):
    """
    Input: integer N: the number of data points (required to be a power of 2?)
    Return: int n: number of qubits needed to represent it (log2(N))
    
    """
    l2 = math.log2(N)
    assert l2 % 1 == 0, "not an integer power of 2"
    return int(l2)


def normalized(v):
    norm = np.linalg.norm(v,2)
    if norm == 0: 
        return v
    return v / norm, norm


In [186]:
def QHomer_Init(data_ft, classicalReg=False):
    """
    Creates an initialization circuit that will create a state with data_ft as coefficients
    Optional: also have a CassicalRegister with equal number of bits
    Input: fourier transformed data: normalized np.array of complex #'s (a,b)
    Return: QuantumCircuit with qreg and optional creg
    
    """
    # number of qubits necessary
    qubit_count = qubits_necessary(len(data_ft))
    #print(f"need {qubit_count} qubits")

    # quantum and classical registers
    qregs = QuantumRegister(qubit_count)
    
    if classicalReg:
        cregs = ClassicalRegister(qubit_count)

    # array of complex amplitudes
    desired_state = data_ft

    # initialization circut
    if classicalReg:
        circuit = QuantumCircuit(qregs, cregs, name="data_loader_trivial")
    else:
        circuit = QuantumCircuit(qregs, name="data_loader_trivial")
        
    circuit.initialize(desired_state, range(qubit_count))
    
    return circuit



In [187]:
def iscft(circ):
    """
    INVERSE semiclassical Fourier Transform.
    Input: QuantumCircuit *without* classical register (for the time being).
    Returns: input circuit with semiclassical FT tacked on.
    
    """

    n = circ.n_qubits
    qc = circ
    # start at highest index qubit
    qubit_range = range(n-1, -1, -1)
    
    for i in qubit_range:
        qc.add_register(ClassicalRegister(1))
    
    for j in qubit_range:
        neighbor_range = range(j-1, -1, -1)
        
        qc.h(j)
        qc.measure(j,j)
        
        for qubit in neighbor_range:
            k = j - qubit
            lam = -math.pi/float(2**(k))
            qc.u1(lam, qubit).c_if(qc.cregs[j],1)
            
    return qc

In [188]:
def multishot_IQFT(data_ft, shots = 20000, noise = False, semiclassical = False):
    """
    Run once for Xs and once for Ys (real and imag)
    args: data normalized np array of either the real or imag components of the ft data
    returns: np array
    """
    
    
    # Make circuit
    if not semiclassical:
        # reverse initialize data because of order convention 
        circ_init = QHomer_Init(data_ft[::-1], classicalReg=True)
        circ_iqft = iqft(circ_init)
        # measure post-qft:
        for j in range(circ_iqft.n_qubits):
            circ_iqft.measure(j, j)
    
    else:
        circ_init = QHomer_Init(data_ft[::-1], classicalReg=False)
        circ_iqft = iscft(circ_init)
        # measure post-qft:
        for j in range(circ_iqft.n_qubits):
            circ_iqft.measure(j, j)
    
    
    
    
    backend = Aer.get_backend('qasm_simulator')
    
    if noise:
        global noise_model
        global coupling_map
        global basis_gates

        
        result = execute(circ_iqft, backend, shots=shots,
                               noise_model=noise_model,
                               coupling_map=coupling_map,
                               basis_gates=basis_gates).result()
        
        counts = result.get_counts(circ_iqft)

        counts = {x.replace(' ', ''): v  
             for x, v in counts.items()} 
        

    else:
        job = execute(circ_iqft, backend, shots=shots)
        counts = job.result().get_counts()
        
        counts = {x.replace(' ', ''): v  
             for x, v in counts.items()}
        
    def make_binary_dict(n):
        """
        creates a dictionary with keys as all possible n-bit binary strings, values = 0
        arg: Int num of bits for binary strings
        returns: Dict  
        """
        binary_tuples = list(it.product(['0', '1'], repeat=n))

        def convertTuple(tup): 
            string =  ''.join(tup) 
            return string

        binary_strings_dict = {}
        for tup in binary_tuples:
            #not sure if this reverse is necessary but I'm not touching anything
            string = convertTuple(tup)[::-1]
            binary_strings_dict[string] = 0
        return binary_strings_dict
    
    n = qubits_necessary(len(data_ft))
    binary_dict = make_binary_dict(n)
    binary_dict.update(counts)   
        
    #make np array
    arr = np.zeros(2**n)
    for key in binary_dict.keys():
        # flip output bits to match
        i = int(key[::-1], 2)
        arr[i] = binary_dict[key]

    return arr/shots







In [215]:
from PIL import Image

def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

def save_image( npdata, outfilename ) :
    img = Image.fromarray( np.asarray( np.clip(npdata,0,255), dtype="uint8"), "L" )
    img.save( outfilename )

In [255]:
# pre-processed to greyscale, 128x128 pixels (multiples of 2)



def process_image(image_name, shots=10000, semiclassical=False, noise=False):
    """
    takes image name string, converts to 2d np array form, outputs new saved image
    """
    image = load_image(image_name)
    (width, height) = image.shape
    new_image = np.empty_like(image)
    
    for row in range(height):
        data = image[row]
        data_ft, norm_factor = normalized(np.fft.fft(data))
        iqft_data = multishot_IQFT(data_ft, shots=shots, semiclassical=semiclassical, noise=noise)
        new_image[row] = (iqft_data*norm_factor).astype(int)
    
    new_name = image_name[:6]
    
    if noise: 
        new_name += '_noisy_'
    else:
        new_name += '_ideal_'
        
    if semiclassical: 
        new_name += '_semiclassicalIQFT_'
    else:
        new_name += '_standardIQFT_'
        
    
    new_name += str(shots) + 'shots.png'
    
    save_image(new_image, new_name)
    
    print('Saved: '+ new_name)
    


In [ ]:
process_image('qiskit_logo_small.png', shots=1000, semiclassical=False, noise=False)
process_image('qiskit_logo_small.png', shots=1000, semiclassical=False, noise=True)
process_image('qiskit_logo_small.png', shots=1000, semiclassical=True, noise=True)

Saved: qiskit_ideal__standardIQFT_1000shots.png


In [239]:
data = np.random.randint(255, size=8)
# data = np.array([1,0,0,0,0,0,0,0])
data_ft, norm_factor = normalized(np.fft.fft(data))
data_ft_ift = normalized(np.fft.ifft(data_ft))

# print('\ndata:')
# print(np.round(data,4))

# print('\ndata_ft:')
# print(np.round(data_ft,4))

# print('\nnewdata:')
# print(np.round(np.real(newdata),4))

shots = 10000
out_ideal = multishot_IQFT(data_ft, shots=shots, semiclassical=False, noise=False)
out_ideal_sc = multishot_IQFT(data_ft, shots=shots, semiclassical=True, noise=False)
out_noise = multishot_IQFT(data_ft, shots=shots, semiclassical=False, noise=True)
out_noise_sc = multishot_IQFT(data_ft, shots=shots, semiclassical=True, noise=True)




In [247]:
print(data)
print(np.round(out_ideal,4))
print(np.round(out_ideal_sc,4))
print((out_noise*norm_factor).astype(int))
print((out_noise_sc*norm_factor).astype(int))

[ 79 228 234 238 207 184  13 245]
[0.0203 0.1727 0.1733 0.1758 0.1408 0.1094 0.0003 0.2074]
[0.0169 0.1679 0.1776 0.1851 0.1451 0.1101 0.0005 0.1968]
[180 196 204 208 187 191 187 209]
[155 215 208 213 194 203 157 218]


In [235]:
image = load_image('qiskit_logo_small.png')

array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.]])